# PYTHON CON CPLEX

<table><tr>
<td> <img src="img/python.png" alt="Drawing" style="width: 220px;"/> </td>
<td> <img src="img/add.png" alt="Drawing" style="width:150px;"/> </td>
<td> <img src="img/cplex.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="img/equal.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="img/laptop.png" alt="Drawing" style="width: 200px;"/> </td>
</tr></table>

En esta clase:
- Descarga de CPLEX (Versión 12.9)
- Conectar CPLEX con PYTHON
- Repaso `Exceptions` en Python
- Ejemplo
- Ejercicio

## Descargar CPLEX

Desde comienzo de este año ya no existe una versión 100% gratis con todas las funcionalidades para estudiantes. Aún asi se puede descargar la versión trial desde [aca](https://www.ibm.com/products/ilog-cplex-optimization-studio).

### CONECTAR CPLEX CON PYTHON:

Requisitos:

1) Tener instalado Cplex:
    - Considere lo siguiente:
        - Si tienes la versión IBM ILOG CPLEX 12.7.1 -> Se puede instalar en python 3.5 y 2.7
        - Si tienes la versióin IBM ILOG CPLEX 12.8 -> Se puede instalar en python 3.5 y 3.6
        - Si tienes la versión IBM ILOG CPLEX 12.9.1 -> Se puede instalar en python 3.7

2) Tener instalado algun gestor de python:
    - Anaconda
    - PIP

Ejemplo con IBM ILOG CPLEX 12.7.1 Y python 3.5 (anaconda) en Windows

1) Crear ambiente de python 3.5(Si es que no tienes uno):
	- Abrir Anaconda Prompt como ADMINISTRADOR
	- Tipiar: conda create -n py35 python=3.5
	- Tipiar: activate py35

2) Ubicar archivo setup.py:
	- Este archivo se encuentra en la dirección donde cplex esta instalado.
	- Ejemplo: C:\Program Files\IBM\ILOG\CPLEX_Studio1271\cplex\python\3.5\x64_win64
	- Ubicar dirección del archivo y copiarla

3) Conectar Cplex con python:
	- Volver al anaconda prompt y tipiar: cd "Ubicación archivo setup.py(Pegar dirección anterior)"
	- Tipiar: python setup.py install

Y LISTO!! 

## Veamos lo que son las excepciones

En palabras simples nos sirven para verificar si alguna función o calculo se puede aplicar, sin que el programa se detenga, de otra forma al encontrar un problema el programa terminaría y las lineas siguientes no serían ejecutadas.

In [ ]:
import sys

randomList = ['a', 0, 2]

for entry in randomList:
    try:
        print("La entrada es", entry)
        r = 1/int(entry)
        break
    except:
        print("Oops! Problema",sys.exc_info()[0],"encontrado")
        print("Siguiente entrada.")
        print()
print("El recíploco de",entry,"es",r)

## Ahora vamos a ver como usarlo

### Estructura de la API

```
CPLEX  
│
└───cplex._internal
│   │   cplex._internal._anno: __Annotation API__
│   │   cplex._internal._constants
|   |   cplex._internal._matrices
|   |   cplex._internal._parameter_classes: Parameters for the 
|   |                                       CPLEX Python API.
│   │   cplex._internal._parameters_auto
│   │   cplex._internal._procedural
│   │   cplex._internal._pwl: __Piecewise Linear API__
│   │   cplex._internal._pycplex
│   │   cplex._internal._subinterfaces
│   
└───cplex.callbacks: Callback classes for the CPLEX Python API.
│   │   Error codes and Exceptions raised by the CPLEX Python API.
│
└───cplex.exceptions.error_codes: Error codes returned by the 
|                                 Callable Library.
│   
└───Classes
|    │   SparsePair: A class for storing sparse vector data.
|    │   SparseTriple: A class for storing sparse matrix data.
|
└───Cplex: A class encapsulating a CPLEX Problem.
|
└───Functions
|    |   terminate(*args, **kwargs): Gracefully stops a CPLEX algorithm.
|
└───Variables
     |   infinity = 1e+20
```

## Probemos con un problema:

$Max~z=x_1+2x_2+3x_3$

sujeto a:

$-x_1 + x_2 + x_3 \leq 20$

$x_1 - 3 x_2 + x_3 \leq 30$

$x_1 \leq 40$

$x_1,x_2,x_3 \geq 0$

In [1]:
import cplex
from cplex.exceptions import CplexError

In [2]:
# Datos comunes para los 3 tipos de alimentación
my_obj = [1.0, 2.0, 3.0]
my_ub = [40.0, cplex.infinity, cplex.infinity]
#my_lb = [0.0, 0.0, 0.0] se omite ya que el valor 0.0 viene por default
my_colnames = ["x1", "x2", "x3"] #importantisimo para poder llamarlas después
my_rhs = [20.0, 30.0]
my_rownames = ["c1", "c2"] #util para obtener información de columnas
my_sense = "LL" # sentido de restricciones, en este caso menor o igual

In [14]:
def populatebyrow(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    # como los lower bounds son todos 0.0 (el default), lb es omitido aca
    prob.variables.add(obj=my_obj, ub=my_ub, names=my_colnames)

    # puede consultar variables de las siguientes formas:

    # lbs es una lista con todos los lower bounds
    lbs = prob.variables.get_lower_bounds()

    # ub1 es el primer lower bound
    ub1 = prob.variables.get_upper_bounds(0)

    # los nombres son ["x1", "x3"]
    names = prob.variables.get_names([0, 2])

    rows = [[[0, "x2", "x3"], [-1.0, 1.0, 1.0]],
            [["x1", 1, 2], [1.0, -3.0, 1.0]]]
    
    
    # rows igual se puede generar como dos listas una con los nombres
    # y la otra con los valores
    #rows = [[0, "x2", "x3"],["x1", 1, 2]]
    #vals = [[-1.0, 1.0, 1.0],[1.0, -3.0, 1.0]]
    
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    # utilizando row y val, por medio de cplex.SparsePair
    #for restriction in range(len(row)):
    #    prob.linear_constraints.add(lin_expr = [cplex.SparsePair(ind = rows[restriction], val=vals[restriction])], 
    #                                senses= [my_sense[restriction]], 
    #                                rhs= [my_rhs[restriction]], 
    #                                names=[my_rownames[restriction]])
    
    # debido a que hay dos argumentos, se toman para especificar un rango, 
    # por lo tanto, cols es toda la matriz de restricciones como una lista de vectores de columna.
    cols = prob.variables.get_cols("x1", "x3")

In [4]:
def populatebycolumn(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                names=my_rownames)

    c = [[[0, 1], [-1.0, 1.0]],
         [["c1", 1], [1.0, -3.0]],
         [[0, "c2"], [1.0, 1.0]]]

    prob.variables.add(obj=my_obj, ub=my_ub, names=my_colnames,
                       columns=c)

In [5]:
def populatebynonzero(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                names=my_rownames)
    prob.variables.add(obj=my_obj, ub=my_ub, names=my_colnames)

    rows = [0, 0, 0, 1, 1, 1]
    cols = [0, 1, 2, 0, 1, 2]
    vals = [-1.0, 1.0, 1.0, 1.0, -3.0, 1.0]

    prob.linear_constraints.set_coefficients(zip(rows, cols, vals))
    # tambien se puede agregar un coeficiente a la vez, ejemplo: row:1 ("x2"), col:1 ("c2"), val:-3.0
    # prob.linear_constraints.set_coefficients(1,1,-3.0)
    # o entregarla en una lista en tripletas
    # prob.linear_constraints.set_coefficients([(0,1,1.0), (1,1,-3.0)])

In [16]:
def lpex1(pop_method):
    try:
        my_prob = cplex.Cplex()

        if pop_method == "r":
            handle = populatebyrow(my_prob)
        elif pop_method == "c":
            handle = populatebycolumn(my_prob)
        elif pop_method == "n":
            handle = populatebynonzero(my_prob)
        else:
            raise ValueError('pop_method debe ser "r", "c" o "n"')

        my_prob.solve()
        
    except CplexError as exc:
        print(exc)
        return

    numrows = my_prob.linear_constraints.get_num()
    numcols = my_prob.variables.get_num()

    print("-------------------------")
    # solution.get_status() retorna un codigo en numero entero
    print("Estado de la Solución = ", my_prob.solution.get_status(), ":", end=' ')
    # la siguiente linea imprime el string correspondiente
    print(my_prob.solution.status[my_prob.solution.get_status()])
    print("Valor objetivo de la Solución  = ", my_prob.solution.get_objective_value())
    slack = my_prob.solution.get_linear_slacks()
    pi = my_prob.solution.get_dual_values()
    x = my_prob.solution.get_values()
    dj = my_prob.solution.get_reduced_costs()
    for i in range(numrows):
        print("Fila %d:  Superavit = %10f  Variable Dual = %10f" % (i, slack[i], pi[i]))
    for j in range(numcols):
        print("Columna %d:  Valor = %10f Costo Reducido = %10f" %
              (j, x[j], dj[j]))
    
    ############## Aca lo guardamos
    
    my_prob.write("lpex1.lp") 

In [15]:
lpex1("r") #r: generar problema por fila

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           202.500000
-------------------------
Solution status =  1 : optimal
Solution value  =  202.5
Row 0:  Slack =   0.000000  Pi =   2.750000
Row 1:  Slack =   0.000000  Pi =   0.250000
Column 0:  Value =  40.000000 Reduced cost =   3.500000
Column 1:  Value =  17.500000 Reduced cost =  -0.000000
Column 2:  Value =  42.500000 Reduced cost =  -0.000000


In [17]:
lpex1("c") #c: generar el problema por columna

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           202.500000
-------------------------
Estado de la Solución =  1 : optimal
Valor objetivo de la Solución  =  202.5
Fila 0:  Superavit =   0.000000  Variable Dual =   2.750000
Fila 1:  Superavit =   0.000000  Variable Dual =   0.250000
Columna 0:  Valor =  40.000000 Costo Reducido =   3.500000
Columna 1:  Valor =  17.500000 Costo Reducido =  -0.000000
Columna 2:  Valor =  42.500000 Costo Reducido =  -0.000000


In [19]:
lpex1("n") #n: generar el problema por "nonzeros"

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           202.500000
-------------------------
Estado de la Solución =  1 : optimal
Valor objetivo de la Solución  =  202.5
Fila 0:  Superavit =   0.000000  Variable Dual =   2.750000
Fila 1:  Superavit =   0.000000  Variable Dual =   0.250000
Columna 0:  Valor =  40.000000 Costo Reducido =   3.500000
Columna 1:  Valor =  17.500000 Costo Reducido =  -0.000000
Columna 2:  Valor =  42.500000 Costo Reducido =  -0.000000


### Pasemos a un MIP
$Max~z= x_1 + 2 x_2 + 3 x_3 + x_4$

Subjeto a:

$- x_1 +   x_2 + x_3 + 10 x_4 \leq 20$

$x_1 - 3 x_2 + x_3      \leq 30$

$x_2      - 3.5x_4  = 0$

$x_1 <= 40$ 

$x_4 <= 3$

$2 <= x_4$

$x_1,x_2,x_3 \geq 0$

$ x_4 \in \mathbb{Z}$
      

In [21]:
my_obj = [1.0, 2.0, 3.0, 1.0]
my_ub = [40.0, cplex.infinity, cplex.infinity, 3.0]
my_lb = [0.0, 0.0, 0.0, 2.0]
my_ctype = "CCCI"
my_colnames = ["x1", "x2", "x3", "x4"]
my_rhs = [20.0, 30.0, 0.0]
my_rownames = ["r1", "r2", "r3"]
my_sense = "LLE"

In [22]:
def populatebyrow(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                       names=my_colnames)

    rows = [[["x1", "x2", "x3", "x4"], [-1.0, 1.0, 1.0, 10.0]],
            [["x1", "x2", "x3"], [1.0, -3.0, 1.0]],
            [["x2", "x4"], [1.0, -3.5]]]

    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)

In [23]:
try:
    my_prob = cplex.Cplex()
    handle = populatebyrow(my_prob)    
    my_prob.solve()
except CplexError as exc:
    print(exc)
    exit()

print("----------------------------")
# solution.get_status() Retorna un codigo en enteros
print("Estado de la Solución = ", my_prob.solution.get_status(), ":", end=' ')
# la siguiente linea imprime el string correspondiente
print(my_prob.solution.status[my_prob.solution.get_status()])
print("Valor de la solución  = ", my_prob.solution.get_objective_value())

numcols = my_prob.variables.get_num()
numrows = my_prob.linear_constraints.get_num()

slack = my_prob.solution.get_linear_slacks()
x = my_prob.solution.get_values()

for j in range(numrows):
    print("Fila %d:  Superavit = %10f" % (j, slack[j]))
for j in range(numcols):
    print("Columna %d:  Valor = %10f" % (j, x[j]))

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 46.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
Aggregator did 1 substitutions.
Reduced MIP has 2 rows, 3 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 2 rows, 3 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           46.0000      163.0000           254.35%
      0     0      125.2083     1       46.0000      125.2083        3  172.

# Ejercicio: Orienteering problem (OP)

Recall the MIP formulation:

In the OP, a set of N vertices $i$ is given, each with a score $S_{i}$. The starting point (vertex 1) and the end point (vertex $N$) are fixed. The time $t_{ij}$ needed to travel from vertex $i$ to $j$ is known for all vertices. Not all vertices can be visited since the available time is limited to a given time budget $T_{max}$. The goal of the OP is to determine a path, limited by Tmax, that visits some of the vertices, in order to maxi- mise the total collected score. The scores are assumed to be entirely additive and each vertex can be visited at most once.
The OP can also be defined with the aid of a graph $G=(V,A)$ where $V=\{v_{1}, . . . , v_{N}\}$ is the vertex set and $A$ is the arc set. In this definition the nonnegative score $S_i$ is associated with each vertex $v_i$ 2 $V$ and the travel time $t_{ij}$ is associated with each arc $a_{ij}$ $A$. The OP consists of determining a Hamiltonian path $G^{’}$ $(G)$ over a sub-set of $V$, including preset start ($v_1$) and end ($v_N$) vertex, and having a length not exceeding $T_{max}$, in order to maximise the total collected score. In most cases, the OP is defined as a path to be found between distinct vertices, rather than a circuit or tour $(v_1,...,v_N)$. In many applications, however, $v_1$ does coincide with $v_N$. The difference between both formulations is small. It is always possible to add a dummy arc between end and start vertex to turn a path problem into a circuit problem. Mansini et al. (2006) explicitly define the ‘‘tour orienteering problem” as an OP where the start and end vertex coincide.
Making use of the notation introduced above, the OP can be for- mulated as an integer problem. The following decision variables are used: $x_{ij} = 1$ if a visit to vertex $i$ is followed by a visit to vertex $j$ – 0 otherwise; $u_i$ = the position of vertex i in the path.

<img src="http://www.sciweavers.org/tex2img.php?eq=Max%20%5Csum_%7Bi%3D2%7D%5E%7BN-1%7D%5Csum_%7Bj%3D2%7D%5E%7BN%7DS_i%20x_%7Bij%7D%20%5C%5C%20%5Csum_%7Bj%3D2%7D%5E%7BN%7Dx_%7B1j%7D%20%3D%20%5Csum_%7Bi%3D1%7D%5E%7BN-1%7D%20x_%7BiN%7D%20%3D%201%5C%5C%0A%5Csum_%7Bi%3D1%7D%5E%7BN-1%7Dx_%7Bik%7D%3D%5Csum_%7Bj%3D2%7D%5E%7BN%7Dx_%7Bkj%7D%20%5Cleq%201%2C%5Cforall%20k%3D2%2C...%2CN-1%5C%5C%0A%5Csum_%7Bi%3D1%7D%5E%7BN-1%7D%5Csum_%7Bj%3D2%7D%5E%7BN%7Dt_%7Bij%7Dx_%7Bij%7D%20%5Cleq%20T_%7Bmax%7D%5C%5C%202%20%5Cleq%20u_%7Bi%7D%20%5Cleq%20N%2C%5Cforall%20i%3D2%2C...%2CN%5C%5C%0Au_%7Bi%7D-u_%7Bj%7D%2B%201%20%5Cleq%20%28N-1%29%281-x_%7Bij%7D%29%2C%5Cforall%20i%2Cj%3D2%2C...%2CN%5C%5C%0Ax_%7Bij%7D%20%5Cin%20%7B0%2C1%7D%2C%5Cforall%20i%2Cj%3D1%2C...%2CN&bc=White&fc=Black&im=jpg&fs=12&ff=txfonts&edit=0" align="center" border="0" alt="Max \sum_{i=2}^{N-1}\sum_{j=2}^{N}S_i x_{ij} \\ \sum_{j=2}^{N}x_{1j} = \sum_{i=1}^{N-1} x_{iN} = 1\\\sum_{i=1}^{N-1}x_{ik}=\sum_{j=2}^{N}x_{kj} \leq 1,\forall k=2,...,N-1\\\sum_{i=1}^{N-1}\sum_{j=2}^{N}t_{ij}x_{ij} \leq T_{max}\\ 2 \leq u_{i} \leq N,\forall i=2,...,N\\u_{i}-u_{j}+ 1 \leq (N-1)(1-x_{ij}),\forall i,j=2,...,N\\x_{ij} \in {0,1},\forall i,j=1,...,N" width="300" height="296" />

The objective function is to maximise the total collected score. The first Constraints guarantee that the path starts in vertex 1 and ends in vertex N. The Second Constraints ensure the connectivity of the path and guarantee that every vertex is visited at most once. Constraint (3) ensures the limited time budget. Constraints (4) and (5) are nec- essary to prevent subtours. These subtour elimination constraints are formulated according to the Miller–Tucker–Zemlin (MTZ) for- mulation of the TSP (Miller et al., 1960).

[Pagina para instancias de OP](https://www.mech.kuleuven.be/en/cib/op#section-2)

# Lectura de Datos y Análisis

Leo los datos desde un txt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ruta = 'data/tsiligirides_problem_1_budget_85.txt'
archivo = pd.read_csv(ruta, header=None, delim_whitespace=True)

Asigno el tiempo o distancia máxima de la ruta

In [ ]:
tiempo_max=archivo.iloc[-1].values
tiempo_max=int(tiempo_max[0])

Creo matriz de distancias

In [ ]:
dist=[]
for i in range(31):
    a=archivo.iloc[i]
    a=np.array(a)
    a=np.delete(a,2)
    for j in range(31):
        if i!=j:
            b=archivo.iloc[j]
            b=np.array(b)
            b=np.delete(b,2)
            dist.append(int(np.linalg.norm(a-b)))
        elif i==j:
            dist.append(99999)

In [ ]:
distancias=np.array(dist)
distancias=distancias.reshape(31,31)
#print(distancias)

Obtengo los beneficios de cada nodo

In [ ]:
beneficios=[]
for i in range(31):
    c= archivo.iloc[i]
    c = [int(i) for i in c]
    del c[0]
    del c[0]
    c=c[0]
    beneficios.append(c)
print(beneficios)   

### EJERCICIO-> HACER LA IMPLEMENTACIÓN EN CPLEX CONECTADO A PYTHON

## Material en internet:

- [Curso de profesor Sergio Correa - Universidad de la Serena](https://www.youtube.com/channel/UCGyGH1nuNrR1C35kWN9BMAg/videos)
- [Documentación oficial (en inglés)](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/refpythoncplex/html/cplex.Cplex-class.html)